# Introduction

### What is Call of Duty/Warzone?

Call of Duty is a very popular video game series published by Activision. Recently, its free-to-play game Warzone has come into great popularity, specifically with the rise of Battle Royale type games. With this rise has also come a very large community that is very competitive. These players started to realize that the game's lack of a ranking system did not match up with their own matchmaking experience. Thus, players have started to wonder if there is a hidden skill-based matchmaking system present in the game.

### Why would someone care about their matchmaking?

In general, all gamers care about their gaming experience. It's obviously not fun to consistently lose, but its also not fun to consistently win. Finding the balance, is very important to staying interested in a game for a long time. So, both from a consumer and developer standpoint, matchmaking is integral to keeping a video game relevant. While this is the case, not being able to see your performance in relation to matchmaking is removing a significant part of the experience from players. In other very popular video games such as League of Legends, Apex Legends, Valorant, and even FIFA all show their players their ranking and progression through the ranked tier system. In Warzone, such a system does not exist, and this leads players to question the skill levels of their opponents and themselves.

 In addition, there is also the possibility that Activision has a financial incentive to modify matchmaking, especially at the content creator level. The reasoning behind this is that content creators hold great influence over potential customers, and giving them a good experience might lead to more customers.

### What exactly is skill based matchmaking?

Skill based matchmaking is a system that matches players in a game based on some ranking. This ranking can be whatever metric the developers choose, but is often implemented as an ELO score or a custom MMR (matchmaking rating) score fitting the game's qualities.

### What is the goal of our project?

We will be exploring two main questions for this project. 

The first question will be "Is there skill-based matchmaking in Call of Duty: Warzone?". This question is a very common one amongst the COD user base and being members of this user base, we wanted to find out an answer.

The second question is arguably juicier because it puts Activision in the hot seat. We will be trying to answer "Does Activision purposefully lower the matchmaking difficulty of content creators?".  

Due to the fact that the game has a very passionate community, learning the answers to these questions can be very insightful for the input fans want to give to developers. For the content creators, players who are also passionate viewers of COD on Twitch or YouTube might rethink their opinions of whoever they watch.

# Data Collection

Due to the specific nature of this project, we had to find creative ways to collect data regarding Call of Duty matchmaking information. Luckily, there is a Call of Duty API that enables developers to look at past match data and stats for specific players. However, Call of Duty's official API enforces a setting where accounts must set their visibility to public for their profiles to be viewable. However, some third party APIs aggregate data across games and paint a clearer picture of players' statistics. One of these is [WZStats.gg](http://wzstats.gg) (Warzone Stats) and they show detailed per match data. By using their website and its API, we are able to get the data of many players that will help to inform our research question. 

Because it is a manual process to set your profile to public visibility, it is likely that better-skilled players are going to be the ones with visible profiles. This potentially limits our visibility into the player skill spectrum. If there is skill-based matchmaking, the initial accounts, and their respective game history, that we are analyzing, will be biased towards higher tier skill levels because these players care a lot more about their stats than lower-skilled players and would be more likely to set their profiles to public. However, if there is no skill-based matchmaking, then the game lobbies will be entirely random as far as skill is concerned (there could be other factors such as network latency and global location). With random lobbies, we will hypothetically be able to tap into the entire spectrum of players if we analyze enough games.

As said, there is no existing database, so we needed to write code that could help create one. In order to do so, we first started by assembling a list of profiles that had public visibility. This included some of our own accounts and also those of pro players and content creators. As mentioned previously, looking at content creators' accounts could provide some insight into our second question, whether the matchmaking skill level of content creators' lobbies were lower. Once we had our profiles picked, we then put them in a csv which our python script would then read and write their lifetime stats, last 100 game stats, and the identifiers of their last 20 games. We then would take these identifiers and scrape the performance data of all players in all of these games. In total, we had  """" 6 accounts, 20 games each """", which adds up to """" 120 games """". In each game, the player cap is 150, so this leads to a upper bound of 18,000 players, but there is of course going to be some overlap in players between games.

The following is the script that got user data, as mentioned first in the paragraph above:

In [ ]:
from csv import writer, reader
from utils import wzstats as wz
import pandas as pd

tasks = []

with open('./config/accounts.csv', 'r') as read:
    f = reader(read)
    header = next(f)

    if header != None:
        for row in f:
            tasks.append((row[0], row[1]))

for t in tasks:
    name = t[0]
    plat = t[1]
    kd = wz.getKD(name, plat)
    winpct = wz.getWinPct(name, plat)
    wins = wz.getWins(name, plat)
    kills = wz.getKills(name, plat)
    killsPerGame = wz.getKillsPerGame(name, plat)
    gulagLast100 = wz.getGulagLast100(name, plat)
    hsLast100 = wz.getHSLast100(name, plat)
    kdLast100 = wz.getKDLast100(name, plat)
    gameIDs = wz.getLast20Matches(name, plat)
    gameIDs = ";".join(gameIDs)

    result = [name, plat, kd, winpct, wins, kills, killsPerGame, gulagLast100, hsLast100, kdLast100, gameIDs]

    with open('./dataset/users.csv', 'a', newline='') as f:
        _writer = writer(f)
        _writer.writerow(result)
        f.close()

This script is the data that iterate through all match identifiers that were scraped in the above code, then added that match data to a file games.csv.

In [ ]:
import utils.wzstats as wz
import csv
import pandas as pd

df = pd.read_csv('./dataset/users.csv')

for index, x in df.iterrows():
    game_ids = x.gameIDs.split(';')
    with open('./dataset/games.csv', 'a', newline='') as csvfile:   
        writer = csv.writer(csvfile, delimiter=',', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
        
        for id in game_ids:
            lobby = wz.getLobbyStats(str(id))
            for player in lobby:
                player['DMG'] = player['DMG'].replace(',', '')
                line = list(player.values())
                writer.writerow(line)

Each of these scripts implement "wzstats.py" which is the script we wrote that implements scraping data from wzstats.gg, the data aggregating site mentioned previously in this writeup. The following code is contained within wzstats.py:

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Xbox = xbox
# Battle.net = battle
# Playstation = psn

XBOX = 'xbl'
BNET = 'battle'
PSN = 'psn'

def getKD(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    return r.json()['data']['lifetime']['mode']['br']['properties']['kdRatio']

def getWins(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return j['data']['lifetime']['mode']['br']['properties']['wins']

def getWinPct(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return ((j['data']['lifetime']['mode']['br']['properties']['wins'] / j['data']['lifetime']['mode']['br']['properties']['gamesPlayed']) * 100)

def getKills(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return j['data']['lifetime']['mode']['br']['properties']['kills']

def getKillsPerGame(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return (j['data']['lifetime']['mode']['br']['properties']['kills'] / j['data']['lifetime']['mode']['br']['properties']['gamesPlayed'])

def getGulagLast100(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return j['last100games']['gulagWinPercentage']

def getHSLast100(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return (j['last100games']['headshots'] / j['last100games']['kills'])

def getKDLast100(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player', params=params)

    j = r.json()

    return (j['last100games']['kills'] / j['last100games']['deaths'])

def getLast20Matches(user, platform):
    params = {
        'username': user,
        'platform': platform
    }

    r = requests.get('https://app.wzstats.gg/v2/player/match', params=params)

    j = r.json()
    
    matches = []

    for m in j:
        matches.append(m['id'])

    return matches

def getAvgKDMatch(match):
    params = {
        'matchId': match
    }

    r = requests.get('https://app.wzstats.gg/v2/', params=params)

    j = r.json()

    return j['matchStatData']['playerAverage']

def getMedianKDMatch(match):
    params = {
        'matchId': match
    }

    r = requests.get('https://app.wzstats.gg/v2/', params=params)

    j = r.json()

    return j['matchStatData']['playerMedian']

def getAvgTeamKDMatch(match):
    params = {
        'matchId': match
    }

    r = requests.get('https://app.wzstats.gg/v2/', params=params)

    j = r.json()

    return j['matchStatData']['teamAverage']

def getMedianTeamKDMatch(match):
    params = {
        'matchId': match
    }

    r = requests.get('https://app.wzstats.gg/v2/', params=params)

    j = r.json()

    return j['matchStatData']['teamMedian']

def getLobbyStats(match):
    # driver = webdriver.Chrome('./chromedriver.exe')
    driver = webdriver.Chrome('/Users/sandeep/Workspace/chromedriver')
    driver.get('https://wzstats.gg/match/' + match)
    time.sleep(10)
    source = driver.page_source
    driver.close()

    soup = BeautifulSoup(source, 'html.parser')
    entries = soup.find_all('div', {'class':'team-container'})

    total = []

    for e in entries:
        container = e.find('div', {'class':'team-players-info-container table-cell'})
        players = container.find_all('div', {'class':'team-players-info-table'})[1:]
        for p in players:
            stats = p.find_all('div', {'class':'team-players-info-cell'})
            kills = stats[0].find('div', {'class':'stat-value'}).text.strip()
            kd = stats[1].find('div', {'class':'stat-value'}).text.strip()
            dmg = stats[2].find('div', {'class':'stat-value'}).text.strip()
            deaths = stats[3].text.strip()
            headshot = stats[4].text.strip()
            result = {'ID': match, 'Kills': kills, 'KD': kd, 'DMG': dmg, 'Deaths': deaths, 'Headshot': headshot}
            total.append(result)

    return total


The following is a snippet of each of our datasets that we built.

In [1]:
import pandas as pd 

# Users

df = pd.read_csv('./dataset/users.csv')
df.head()

,name,plat,kd,winpct,wins,kills,killsPerGame,gulagLast100,hsLast100,kdLast100,gameIDs
0,d0ncic,xbl,1.234568,3.333333,5,500,3.333333,0.619048,0.188841,1.153465,127364889490544550;6320684864520723223;1033676...
1,NICKMERCS#11526,battle,4.373025,14.764834,744,46494,9.226831,0.827586,0.208376,4.429864,17837158414789481963;12566442349070955611;2833...
2,AYDAN#11691,battle,4.982685,12.710220,786,69927,11.307730,0.880952,0.171028,4.734513,1992934777753343208;7495523342168615093;734690...
3,TEEPEE#1840,battle,5.094419,27.636719,1981,81149,11.321010,0.879121,0.219237,5.342593,7281667249466132419;11172228004734822107;17832...
4,TOMMEY#21329,battle,5.056241,20.023945,1338,88105,13.185424,0.825000,0.267717,4.861244,7505873604023675579;4894258733829837567;224560...


In [2]:
df = pd.read_csv('./dataset/games.csv')
df.head()

,id,kills,kd,dmg,deaths,hsPct
0,127364889490544550,10,1.53,3710,0,10%
1,127364889490544550,6,0.75,2081,2,100%
2,127364889490544550,1,0.47,663,2,0%
3,127364889490544550,8,1.58,2744,5,38%
4,127364889490544550,3,0.89,1568,1,33%


# Data Analysis